In [1]:
import casadi as ca
import numpy as np

def create_xtrack_fun()->ca.Function:
    a = ca.SX.sym('a', 2)
    b = ca.SX.sym('b', 2)
    pos = ca.SX.sym('pos', 2)
    ba = (b - a)
    ba_n = ba / ca.norm_2(ba)    
    distance = pos - (a + (ba_n.T @ (pos - a)) * ba_n)
    return ca.Function('xtrack', [a, b, pos], [ca.norm_2(distance)])
    
x_track = create_xtrack_fun()
x_track.call([[0,0], [10,1], [1,2]])


[DM(1.89057)]

In [2]:
A = ca.SX.sym('A', 3)

x = ca.SX.sym('x')

b = A +  ca.vertcat(x, 2 * ca.cos(x), ca.cos(x))
b

SX([(A_0+x), (A_1+(2*cos(x))), (A_2+cos(x))])

In [3]:
A = ca.SX.sym('A',10, 3)
print(A)
A[0, :2]


[[A_0, A_10, A_20], 
 [A_1, A_11, A_21], 
 [A_2, A_12, A_22], 
 [A_3, A_13, A_23], 
 [A_4, A_14, A_24], 
 [A_5, A_15, A_25], 
 [A_6, A_16, A_26], 
 [A_7, A_17, A_27], 
 [A_8, A_18, A_28], 
 [A_9, A_19, A_29]]


SX([[A_0, A_10]])

In [4]:
import numpy as np
states = np.zeros((10,4))
states[0]
d = ca.DM([i for i in range(10)])
print(d)
d[:-1] = d[1:]
print(d)


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 9]


In [5]:
A = ca.SX.sym('A', 3,2)
x = A[0,:]
y = A[1,:].T

f = ca.Function("test", [x, y], [x @ y])
# print(A, ca.sqrt(x[0]**2 + x[1]**2))
f([1,0], [10,0])

DM(10)

In [6]:
phi = ca.MX.sym('phi')

# Вектор, содержащий cos(phi) и sin(phi)
vector = ca.horzcat(ca.cos(phi), ca.sin(phi))
vector.shape


(1, 2)

In [7]:
from mpc.mpc_casadi import MPCCasadi
import math
A = ca.SX.sym('A', 2).T
B = ca.SX.sym('B', 2).T
POS = ca.SX.sym('POS', 2).T
HEADING = ca.SX.sym('HEADING')
print(f'A:{A.shape} B:{B.shape}')

xtrack, heading_errro, t = MPCCasadi.get_track_values(A, B, POS, HEADING)
t_fun = ca.Function('t', [A,B, POS, HEADING], [t])
t_xtrack = ca.Function('xtrack', [A,B, POS, HEADING], [xtrack])
t_heading_errro = ca.Function('heading_errro', [A,B, POS, HEADING], [heading_errro])

state = [[0, 0], [100, 0], [150, -10], math.radians(10)]
print(f"t:{t_fun(state[0], state[1], state[2], state[3])}")
print(f"xtrack:{t_xtrack(state[0], state[1], state[2], state[3])}")
print(f"heading_errro:{math.degrees(t_heading_errro(state[0], state[1], state[2], state[3]))}")


A:(1, 2) B:(1, 2)
t:1.5
xtrack:10
heading_errro:10.000000000000012


In [8]:
from mpc.mpc_casadi import MPCCasadi
import math
path = ca.SX.sym('path', 3, 2)
pos = ca.SX.sym('pos', 2).T
heading = ca.SX.sym('heading')
print(f'A:{A.shape} B:{B.shape}')

xtrack, heading_error = MPCCasadi.get_track_cost(path, pos, heading)

f_xt = ca.Function('f_xt', [path, pos, heading], [xtrack])
f_he = ca.Function('f_he', [path, pos, heading], [heading_error])

path = [[0.,0.], [0., 10.], [10., 10.]]
pos = [1., 2.]
heading = math.radians(0)
print(f"f_xt:{f_xt(path, pos, heading)}")
print(f"f_he:{f_he(path, pos, heading)}")


A:(1, 2) B:(1, 2)


NotImplementedError: Wrong number or type of arguments for overloaded function 'Function_call'.
  Possible prototypes are:
    call(self,dict:DM,bool,bool)
    call(self,[DM],bool,bool)
    call(self,[SX],bool,bool)
    call(self,dict:SX,bool,bool)
    call(self,dict:MX,bool,bool)
    call(self,[MX],bool,bool)
  You have: '(Function,([[float]],[float],float))'


In [ ]:
import casadi as ca
import numpy as np
from typing import List, Tuple

A = ca.SX.sym('A', 1)
b = A * A
test = ca.Function('test', [A,], [b,])
test(2)



def get_distance(polygon:List[Tuple[float, float]], circle:Tuple[Tuple[float, float], float])->float:
    """return distance between polygon and circle. 

    Args:
        poligon (List[Tuple[float, float]]): points of poligon
        circle (Tuple[Tuple[float, float], float]): circle description: ((x,y), radius)
    Return (float): result >= 0 - object not intersected, result < 0 intersected  
    """
    for index in range(len(polygon) - 1):
        point_a = polygon[index]
        point_b = polygon[index + 1]
        ba = point_b - point_a
        normal_ba = ca.horzcat(-ba[1], ba[0]) / ca.norm_2(ba)
        line_offset = normal_ba.T @ point_a
        return line_offset


    

DM(4)